In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.display.max_columns = 100
from google.cloud import bigquery
from datetime import datetime
from datetime import date, timedelta

import sys
sys.path.append('/home/jupyter/aaaie-customer-personas/src')

from customer_personas.bq_utils import export_dataframe_to_bq, upload_data_to_gcs

In [ ]:
client = bigquery.Client(location="northamerica-northeast1")

## cto-wln-sa-data-pr-bb5283

In [ ]:
query = \
'''
SELECT *
FROM `cto-wln-sa-data-pr-bb5283`.INFORMATION_SCHEMA.SCHEMATA
'''

In [ ]:
table_df = client.query(query).to_dataframe()

In [ ]:
table_df.schema_name.unique()

In [ ]:
table_column_query = \
'''
SELECT
  *
FROM
  `{project_id}.{dataset_name}`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
'''

In [ ]:
project_column_info_df = pd.DataFrame()
for index, row in table_df.iterrows():
    query_ = table_column_query.format(project_id = row['catalog_name'],
                                       dataset_name = row['schema_name'])
    df = client.query(query_).to_dataframe()
    project_column_info_df = project_column_info_df.append(df)

In [ ]:
project_column_info_df.reset_index(drop=True)

## Datahub Tables

In [ ]:
datahub_dataset_names = pd.read_csv('datahub_table_names.csv')

In [ ]:
datahub_dataset_names.head(2)

In [ ]:
datahub_column_query = \
'''
SELECT
  *
FROM
  `cio-datahub-enterprise-pr-183a.{dataset_name}`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
'''

In [ ]:
datahub_column_info_df = pd.DataFrame()
for index, row in datahub_dataset_names.iterrows():
    query_ = datahub_column_query.format(dataset_name = row['datahub_table_names'])
    try:
        df = client.query(query_).to_dataframe()
        datahub_column_info_df = datahub_column_info_df.append(df)
    except: 
        continue

In [ ]:
datahub_column_info_df.reset_index(drop=True)

In [ ]:
BQ_TABLE_ID= 'cto-wln-sa-data-pr-bb5283.ref_table.datahub_schema_info'
export_dataframe_to_bq(datahub_column_info_df.reset_index(drop=True),
                       table_id=BQ_TABLE_ID,
                       schema_list=[],
                       generate_schema=True)